In [1]:
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import urllib3
import datetime
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

hardcode = ['7/7', '7/7', '6/7 + Bonus', '6/7', '5/7', '5/7', '4/7', '4/7', '3/7 + Bonus']
size = [(7, False), (7, True), (6, True), (6, False), (5, False), (5, True), (4, False), (4, True), (3, True)]
intersections = dict(zip(size, hardcode))

filename = 'numbers.txt'

def get_dates(date1, date2):
    fridays = []
    d = date1
    while d <= date2:
        if d.weekday() == 4:
            fridays.append(d)
        d += datetime.timedelta(1)
    return fridays

def get_url(year, month, day):
    url_template = 'https://lottery.olg.ca/en-ca/winning-numbers/lotto-max/winning-numbers?'
    #year = str(year)
    startDate = datetime.date(year, month, day) - datetime.timedelta(1)
    endDate = datetime.date(year, month, day) + datetime.timedelta(1)
    #startDate = year + '-' + '{:02d}'.format(month) + '-' + '{:02d}'.format(day - 1)
    #endDate = year + '-' + '{:02d}'.format(month) + '-' + '{:02d}'.format(day + 1)
    url = url_template + 'startDate=' + str(startDate) + '&' + 'endDate=' + str(endDate)
    return url

def get_data(url):
    data = dict()
    http = urllib3.PoolManager()
    r = http.request('GET', url)
    soup = BeautifulSoup(r.data, 'html.parser')
    
    # getting numbers
    nums = soup.findAll('span', attrs={'class': 'winning-numbers-number'})
    win_numbers = []
    for number in nums:
        win_numbers.append(number.text)
    
    # getting money wins from the table
    table = soup.find_all('tbody')[0]
    values = dict()
    for row in table.find_all('tr'):
        value = float(row.find('td', {'class':'chart-prize-col'}).text[1:].replace(',',''))
        winner = row.find('td', {'class':'chart-winners-col'}).text
        values[winner] = value
    
    # getting a date
    current_date = soup.find('p', {'class':'large-date'}).text
    
    ### saving
    data['numbers'] = [int(a) for a in win_numbers[:6]]
    data['bonus'] = int(win_numbers[7])
    data['date'] = current_date
    data['money'] = values
    return data

def total_sum(database, info):
    total = 0
    free_plays = 0
    for numbers in database:
        inters = set(numbers).intersection(set(info['numbers']))
        size = len(inters)
        bonus = info['bonus'] in set(numbers)
        win = 0.0
        free_play = 0
        if (size, bonus) == (3, False):
            free_play = 1
            free_plays += 1
        if (size, bonus) in intersections:
            key = intersections[(size, bonus)]
            win = info['money'][key]
        total += win
        print('You win ${:>6} and {} free play for a ticket with numbers {}'.format(win, free_play, numbers))
    print('\nYour total win on {} is ${} and {} free plays'.format(info['date'], total, free_plays))
    return np.array([total, free_plays])
    
def summary(date1, date2):
    fridays = get_dates(date1, date2)
    super_total = np.array([0., 0.])
    for friday in fridays:
        day = friday.day
        month = friday.month
        year = friday.year
        info = get_data(get_url(year, month, day))

        print('On {} \nwinning numbers are: {} - {}'.format(info['date'], info['numbers'], info['bonus']))
        print('Table of winnings:')
        for key in info['money']:
            print('{:>15} ----- ${}'.format(key, info['money'][key]))

        print()
        moneey = total_sum(database, info)
        print('\n\n---------------------------------------------------')
        super_total += moneey
    print('\n\nTOTAL = ${} and {} free plays'.format(super_total[0], super_total[1]))

In [2]:
# Set up the dates 
year_from = 2019
month_from = 2
day_from = 1

year_to = 2019
month_to = 5
day_to = 1

date1 = datetime.date(year_from, month_from, day_from)
date2 = datetime.date(year_to, month_to, day_to)
print('Your tickets are: ')
database = np.loadtxt(filename, dtype=np.int64)
print(database)
print('\n\n')

summary(date1, date2)

Your tickets are: 
[[ 3  8 17 21 28 34 39]
 [ 1  3  9 12 24 29 36]
 [13 14 17 22 34 36 43]
 [ 1  3 12 16 20 29 30]
 [ 2  6  8 16 21 26 42]
 [26 30 35 38 40 43 47]
 [11 15 27 30 37 43 46]
 [ 9 17 26 28 39 44 47]
 [ 2  8 12 30 35 45 48]
 [ 3 13 17 38 40 42 43]
 [16 21 26 28 31 40 47]
 [13 15 18 28 32 44 47]]



On Fri, Feb 1 
winning numbers are: [2, 6, 13, 19, 21, 32] - 12
Table of winnings:
            7/7 ----- $26600135.8
    6/7 + Bonus ----- $324371.8
            6/7 ----- $4539.0
            5/7 ----- $103.8
            4/7 ----- $20.0
    3/7 + Bonus ----- $20.0

You win $   0.0 and 0 free play for a ticket with numbers [ 3  8 17 21 28 34 39]
You win $   0.0 and 0 free play for a ticket with numbers [ 1  3  9 12 24 29 36]
You win $   0.0 and 0 free play for a ticket with numbers [13 14 17 22 34 36 43]
You win $   0.0 and 0 free play for a ticket with numbers [ 1  3 12 16 20 29 30]
You win $   0.0 and 1 free play for a ticket with numbers [ 2  6  8 16 21 26 42]
You win $   0.0 and

On Fri, Mar 15 
winning numbers are: [6, 11, 25, 31, 36, 41] - 47
Table of winnings:
            7/7 ----- $25841633.7
    6/7 + Bonus ----- $322326.1
            6/7 ----- $5910.2
            5/7 ----- $107.1
            4/7 ----- $20.0
    3/7 + Bonus ----- $20.0

You win $   0.0 and 0 free play for a ticket with numbers [ 3  8 17 21 28 34 39]
You win $   0.0 and 0 free play for a ticket with numbers [ 1  3  9 12 24 29 36]
You win $   0.0 and 0 free play for a ticket with numbers [13 14 17 22 34 36 43]
You win $   0.0 and 0 free play for a ticket with numbers [ 1  3 12 16 20 29 30]
You win $   0.0 and 0 free play for a ticket with numbers [ 2  6  8 16 21 26 42]
You win $   0.0 and 0 free play for a ticket with numbers [26 30 35 38 40 43 47]
You win $   0.0 and 0 free play for a ticket with numbers [11 15 27 30 37 43 46]
You win $   0.0 and 0 free play for a ticket with numbers [ 9 17 26 28 39 44 47]
You win $   0.0 and 0 free play for a ticket with numbers [ 2  8 12 30 35 45 48]
You 